In [10]:
# Single GPU
import sagemaker
from sagemaker.pytorch import PyTorch
from sagemaker.debugger import TensorBoardOutputConfig

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket="tf-binding-sites")

training_data_s3_path = "s3://tf-binding-sites/pretraining/data/"
output_s3_path = "s3://tf-binding-sites/finetuning/results/output"

tensorboard_output_config = TensorBoardOutputConfig(
    s3_output_path="s3://tf-binding-sites/finetuning/results/tensorboard",
    container_local_output_path="/opt/ml/output/tensorboard"
)

use_spot_instances = True
max_wait = 1209600 if use_spot_instances else None
checkpoint_s3_bucket="s3://tf-binding-sites/finetuning/results/checkpointing"
checkpoint_local_path="/opt/ml/checkpoints"


estimator = PyTorch(
    base_job_name="tf-finetuning-no-weights",
    entry_point="tf_prediction.py",
    source_dir="../training",  # Directory containing training script and other files
    output_path=output_s3_path,
    code_location="s3://tf-binding-sites/finetuning/results/code",
    role=role,
    py_version="py310",
    framework_version='2.1.0',
    volume_size=600, 
    instance_count=1,
    max_run=1209600,
    instance_type='ml.g5.8xlarge',
    hyperparameters={
        'learning-rate': 1e-6
    },
    tensorboard_output_config=tensorboard_output_config,
    # use_spot_instances=use_spot_instances,
    # max_wait=max_wait,
    # checkpoint_s3_uri=checkpoint_s3_bucket,
    # checkpoint_local_path=checkpoint_local_path
)

estimator.fit({'training': training_data_s3_path}, wait=False)



sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
Using provided s3_resource


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: tf-finetuning-no-weights-2024-03-12-18-16-01-327


ResourceLimitExceeded: An error occurred (ResourceLimitExceeded) when calling the CreateTrainingJob operation: The account-level service limit 'ml.g5.8xlarge for training job usage' is 4 Instances, with current utilization of 4 Instances and a request delta of 1 Instances. Please use AWS Service Quotas to request an increase for this quota. If AWS Service Quotas is not available, contact AWS support to request an increase for this quota.